In [ ]:
import os
os.chdir('..') #necessary to go one level up to the root directory to find dfm_tools in notebook/binder
import xarray as xr
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
import dfm_tools as dfmt


In [ ]:
#set paths and parameters per model (change 'model' parameters to get different results/plots)
model = 'grevelingen' #'curvedbend' 'grevelingen' 'westernscheldt'

dir_opendap = 'http://opendap.deltares.nl/thredds/dodsC/opendap/deltares/Delft3D/netcdf_example_files'
if model=='curvedbend':
    file_nc_his = dir_opendap + '/DFM_curvedbend_3D/cb_3d_his.nc'
    file_nc_map = [dir_opendap + '/DFM_curvedbend_3D/cb_3d_map.nc'] #list since opendap does not support glob
    layer = 5
    crs = None
    line_array = np.array([[ 104.15421399, 2042.7077107 ],
                           [2913.47878063, 2102.48057382]])
elif model=='grevelingen': #multipart mapfile has to be list in case of opendap
    file_nc_his = dir_opendap + '/DFM_grevelingen_3D/Grevelingen-FM_0000_his.nc'
    file_nc_map = [dir_opendap + f'/DFM_grevelingen_3D/Grevelingen-FM_{i:04d}_map.nc' for i in range(8)]
    layer = 34
    crs = 'EPSG:28992'
    line_array = np.array([[ 53181.96942503, 424270.83361629],
                           [ 55160.15232593, 416913.77136685]])
elif model=='westernscheldt':
    file_nc_his = None
    file_nc_map = [dir_opendap + '/westernscheldt_sph_map.nc'] #list since opendap does not support glob
    layer = None
    crs = 'EPSG:4326'
    line_array = None
else:
    raise Exception(f'undefined model: {model}')


In [ ]:
#open hisfile with xarray and print netcdf structure
if file_nc_his is not None:
    data_xr_his = xr.open_mfdataset([file_nc_his], preprocess=dfmt.preprocess_hisnc)
    print(data_xr_his)
    stations_pd = data_xr_his['stations'].to_dataframe()
    print('\nStations in netcdf dataset:\n',stations_pd[['station_x_coordinate','station_y_coordinate']])


In [ ]:
#plot his data: waterlevel at stations
if file_nc_his is not None:
    fig, ax = plt.subplots(1,1,figsize=(10,5))
    data_xr_his.waterlevel.plot.line(ax=ax, x='time')
    ax.legend(data_xr_his.stations.to_series(),loc=1,fontsize=8) #optional, to change legend location
    fig.tight_layout()


In [ ]:
#plot his data: temperature zt at one station
if file_nc_his is not None:
    data_xr_selzt = data_xr_his.isel(stations=0).isel(time=slice(0,50))
    fig, ax1 = plt.subplots(figsize=(12,6))
    c = dfmt.plot_ztdata(data_xr_sel=data_xr_selzt, varname='salinity', ax=ax1, cmap='jet')
    fig.colorbar(c,ax=ax1,label=f'salinity at {data_xr_selzt.stations.data}')
    CS = dfmt.plot_ztdata(data_xr_sel=data_xr_selzt, varname='salinity', ax=ax1, only_contour=True, levels=6, colors='k', linewidths=0.8, linestyles='solid')
    ax1.clabel(CS, fontsize=10)
    fig.tight_layout()

In [ ]:
#open+merge mapfile with xugrid(xarray) and print netcdf structure
data_xr_mapmerged = dfmt.open_partitioned_dataset(file_nc_map)
print(data_xr_mapmerged)

In [ ]:
#plot net/grid. use random variable and plot line to get grid
fig, ax = plt.subplots(figsize=(10,4))
pc = data_xr_mapmerged.ugrid.grid.plot(edgecolor='crimson', linewidth=0.5)
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)
if line_array is not None:
    ax.plot(line_array[:,0],line_array[:,1],'b',linewidth=3)
fig.tight_layout()


In [ ]:
#plot water level on map
fig, ax = plt.subplots(figsize=(10,4))
pc = data_xr_mapmerged['mesh2d_s1'].isel(time=3).ugrid.plot(edgecolor='face',cmap='jet')
pc.set_clim(-0.5,2)
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)
fig.tight_layout()


In [ ]:
#plot eastward velocities on map
fig, ax = plt.subplots(figsize=(10,4))
if layer is None:
    pc = data_xr_mapmerged['mesh2d_ucx'].isel(time=3).ugrid.plot(edgecolor='face',cmap='jet')
else:
    pc = data_xr_mapmerged['mesh2d_ucx'].isel(time=3,nmesh2d_layer=layer).ugrid.plot(edgecolor='face',cmap='jet')
if crs is None:
    ax.set_aspect('equal')
else:
    ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False)
fig.tight_layout()

In [ ]:
#plot slice/sideview trough 3D salinity mapdata
if line_array is not None:
    xr_crs_ugrid = dfmt.polyline_mapslice(data_xr_mapmerged, line_array, timestep=3)
    fig, ax = plt.subplots()
    xr_crs_ugrid['mesh2d_sa1'].ugrid.plot(cmap='jet')
    fig.tight_layout()
